csv 하나 작업하면 여기에 있는 셀로만 돌린거라면, 다시시작 한번 해주고 다른 csv read해서 쓰세요

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#구글드라이브연동

한국어 유사도 측정을 위해서, 정규표현식으로, 특수기호 및 영어로만 된 문장 제거 <br>
이 후, 문장 내에서 영어의 비율을 계산하고 일정 비율 이상인 문장을 제거, 남아있는 문장에서는 영어 제거<br>


In [ ]:
def strip_lyrics(sample):
    sample_lyrics = [k.strip() for k in sample.split('\n') if k.strip() != '']
    return sample_lyrics

def lyrics_preprocessing(lyrics):
    # 영어로 된 문장 및 영어 비율이 높은 문장 제거
    filtered_lyrics = []
    for line in lyrics:
        if re.search(r'[a-zA-Z]', line):
            continue

        line_without_brackets = re.sub(r'\([^)]*\)', '', line)

        # 영어 비율 계산
        total_chars = len(line_without_brackets.replace(' ', ''))  # Consider only non-space characters
        english_chars = sum(1 for char in line_without_brackets if char.isalpha() and char.isascii())

        # 영어 문자가 없는 경우, 영어 비율은 0으로 설정
        english_ratio = english_chars / total_chars if total_chars != 0 else 0

        if english_ratio >= 0.7:
            continue

        if line_without_brackets.strip() != "":
            filtered_lyrics.append(line_without_brackets.strip())

    result_lyrics = []
    for line in filtered_lyrics:
        line_without_english = ''.join(char for char in line if not char.isalpha() or not char.isascii())
        line_without_special_chars = re.sub(r'[^\w\s가-힣]', '', line_without_english)
        result_lyrics.append(line_without_special_chars.strip())

    unique_result_lyrics = list(set(result_lyrics))
    return unique_result_lyrics

In [ ]:
playlist=pd.read_csv('/content/drive/MyDrive/our_prj/calm0824.csv')

가사 전처리를 하고 나서 남은 결과값이 의미가 없거나 중복이거나, NULL이라면 제거를 합니다.

In [ ]:
# 가사 전처리
playlist['processed_lyrics'] = playlist['lyrics'].apply(strip_lyrics)

for i in range(len(playlist)):
    lyrics = playlist['processed_lyrics'][i]
    processed_lyrics = lyrics_preprocessing(lyrics)


    processed_lyrics_minus_one = []
    for x in processed_lyrics:
        if isinstance(x, (int, float)):
            processed_lyrics_minus_one.append(x - 1)
        else:
            processed_lyrics_minus_one.append(x)

    playlist.at[i, 'processed_lyrics'] = processed_lyrics_minus_one

playlist['processed_lyrics'] = playlist['processed_lyrics'].apply(lambda lyrics: [line for line in lyrics if len(line) >= 10])
playlist = playlist[playlist['processed_lyrics'].apply(len) > 0].reset_index(drop=True)

In [ ]:
sum(playlist['processed_lyrics'].isnull())

In [ ]:
# 각 리스트에서 길이가 10글자 미만인 문장을 찾는 함수
def find_short_sentences(lyrics):
    return [line for line in lyrics if len(line) < 10]
short_sentences_in_playlist = playlist['processed_lyrics'].apply(find_short_sentences)
non_empty_count = len(short_sentences_in_playlist[short_sentences_in_playlist.apply(lambda x: len(x) > 0)])
print("Not null count:", non_empty_count)

중복제거 버전

In [ ]:
playlist.to_csv('calm0914_final.csv')

전처리 완료된 CSV 읽을 때 주의할 점.
1. final 버전 csv를 read_csv 합니다.
2. 해당 csv를 받은 데이터 프레임 변수에 apply 명령어 써놓은 2줄을 사용합니다.
3. 이후, 'processed_lyrics' 열에 있는 데이터를 뽑아 씁니다.

In [ ]:
playlist = pd.read_csv('/content/drive/MyDrive/our_prj/calm0914_final.csv')

In [ ]:
playlist['processed_lyrics'] = playlist['lyrics'].apply(strip_lyrics)
playlist['processed_lyrics_str'] = playlist['processed_lyrics'].apply('\n'.join)

In [ ]:
li=playlist['processed_lyrics'][3]
for i in li:
  print(i)

중복 제거를 했다면?

In [ ]:
import pandas as pd


playlist


sum_all_char=0
sum_all_sentence=0
data_length=0
for index, row in playlist.iterrows():
    sentences = row['processed_lyrics']
    sum=0
    total_length=len(sentences)
    for i in sentences:
      sum+=len(i)
    if sum <1:
      average_sentence_length=0
    else:
      average_sentence_length = sum / total_length
    sum_all_char+=int(average_sentence_length)
    sum_all_sentence+=int(total_length)
    data_length+=1
    print(f"행 {index + 1} - 문자 평균 길이: {average_sentence_length:.2f} / 전체 문장 개수 : {total_length}")

average_total_sentence_count = sum_all_sentence / data_length
average_total_char_length = sum_all_char / data_length

print("\n전체 데이터에 대한 평균 문장 개수:", average_total_sentence_count)
print("전체 데이터에 대한 평균 문자 평균 길이:", average_total_char_length)